Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 2: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [605]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
from copy import deepcopy
from dataclasses import dataclass
import copy
from random import randint


## The *Nim* and *Nimply* classes

In [606]:
Nimply = namedtuple("Nimply", "row, num_objects")
NUM_ROWS = 5


In [607]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects    
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [608]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [609]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [610]:
POPULATION_SIZE = 50
NUM_GENERATIONS = 10

In [611]:
@dataclass
class Individual:
    genome: float
    fitness: int
#Adaptive (1+labmda) - ES

population = [
    Individual(
        genome=0.5,
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

def generate_valid_move(state: Nim, ind: Individual) -> tuple:
    #The row is cho0sen based on genome value
    row = int(ind.genome * (NUM_ROWS - 1))

    #We assure that the move is valid by selecting the next row valid
    while state.rows[row] == 0:
        row = (row+1)%NUM_ROWS

    #We use the genome to determine the number of objects to take
    num_objects = int((ind.genome * (NUM_ROWS - 1)) * state.rows[row]) + 1
    num_objects = max(1, min(num_objects, state.rows[row]))

    return row, num_objects


def generate_child(parent, state: Nim, σ):
    
    child_genome = (
        parent.genome + np.random.normal(loc=0, scale=σ)
    )

    return Individual(genome=child_genome, fitness=None)


def fitness(individual: Individual, state: Nim):
    temp_state = copy.deepcopy(state)
    (row,num_objects) = generate_valid_move(state,individual)
    temp_state.nimming(Nimply(row, num_objects))

    num_rows_not_zero = sum(1 for row in temp_state.rows if row != 0)
    odd_rows_with_one_stick = sum(1 for row in temp_state.rows if num_rows_not_zero % 2 != 0 and row == 1)
    penalty_for_single_stick_rows = sum(1 for row in temp_state.rows if row == 1)
    incentive_for_remaining_sticks = sum(temp_state.rows)
    fitness_score = -odd_rows_with_one_stick + penalty_for_single_stick_rows - incentive_for_remaining_sticks

    return fitness_score



def es(state: Nim):

    λ = 50 
    σ = 0.001 #dev std
    stats = [0,0]
    best_prev_fitness=-9999
    for step in range(NUM_GENERATIONS):
        
        for individual in population:
            individual.fitness = fitness(individual,state)

        
        population.sort(key=lambda x: x.fitness, reverse=True)

        # Select best individual as parent
        parent = population[0]
        # Genera λ figli
        children = [generate_child(parent,state,σ) for _ in range(λ)]
        # Evaluating children fitness
        stats[0] += λ
        for child in children:
            child.fitness = fitness(child,state)
            stats[1] += int(-child.fitness > -best_prev_fitness) #because the fitness is negative value
       
        #stats[0] total number of fitness evaluations
        #stats[1] number of evaluations that are greater than the current one
        if stats[1] != 0 and stats[0] / stats[1] < 1 / 5:
            σ /= 1.1
        elif stats[1] != 0 and stats[0] / stats[1] > 1 / 5:
            σ *= 1.1

        
        population.sort(key=lambda i:i.fitness,reverse=True) 
        population[:-λ]=children

        
        best_prev_fitness=population[0].fitness
   
    best_individual = population[0]
    (row,num_objects) = generate_valid_move(state,best_individual)
    return Nimply(row,num_objects)




In [612]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [613]:
logging.getLogger().setLevel(logging.INFO)

strategy = (gabriele, es)

nim = Nim(5)


logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 5 7 9>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=5)
INFO:root:status: <0 3 0 7 9>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=3)
INFO:root:status: <0 0 0 7 9>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=7)
INFO:root:status: <0 0 0 0 9>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=9)
INFO:root:status: <0 0 0 0 0>
INFO:root:status: Player 1 won!
